In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rads_extraction
import tec_interpolation


- RADS model 
- IMIC model 

In [ ]:
def iono_formula(sat:str, TEC_gps:np.ndarray, alpha:float, beta:float)->np.ndarray:
    if sat == 'c2':
        f = 13.575e9 # TODO update
    elif sat == 'j3':
        f = 13.575e9 # primary band
        # f2 = 5.3e9 # secondary band
    elif sat == 's3':
        f = 13.575e9
    tecu = 1e16
    
    return (40.3/f**2)* alpha*beta* (TEC_gps*tecu)
    

In [2]:
validate = True
sat = 'c2'




time, lat, lon, sla_corrected   = rads_extraction.extract_rads(fr'{sat}_2016_summer.nc') #TODO check filenames
time, lat, lon, sla_uncorrected = rads_extraction.extract_rads(fr'{sat}_2016_summer_no_iono.nc')

sla_corrected-sla_uncorrected


array([0.0519, 0.0078, 0.0272, 0.0225, 0.0095, 0.0224, 0.0112, 0.0126,
       0.0371, 0.0134, 0.0345, 0.0221, 0.0309, 0.0286, 0.0108, 0.0088,
       0.0381, 0.0277, 0.0063, 0.0409, 0.0273, 0.0239, 0.0062, 0.0254,
       0.0066, 0.0545, 0.0079, 0.0186, 0.0093, 0.0395, 0.0343, 0.0085,
       0.0283, 0.035 ])

In [3]:
tec_GPS = tec_interpolation.mass_interpolate(lon, lat, time)

Checking availability of source GIMs...
Downloaded and decompressed: main/temp/jpli1970.16i.nc.gz
All neccessary source GIMs availible for interpolation!
Staring mass interpolation...
Progress:  1 / 34
Progress:  2 / 34
Progress:  3 / 34
Progress:  4 / 34
Progress:  5 / 34
Progress:  6 / 34
Progress:  7 / 34
Progress:  8 / 34
Progress:  9 / 34
Progress:  10 / 34
Progress:  11 / 34
Progress:  12 / 34
Progress:  13 / 34
Progress:  14 / 34
Progress:  15 / 34
Progress:  16 / 34
Progress:  17 / 34
Progress:  18 / 34
Progress:  19 / 34
Progress:  20 / 34
Progress:  21 / 34
Progress:  22 / 34
Progress:  23 / 34
Progress:  24 / 34
Progress:  25 / 34
Progress:  26 / 34
Progress:  27 / 34
Progress:  28 / 34
Progress:  29 / 34
Progress:  30 / 34
Progress:  31 / 34
Progress:  32 / 34
Progress:  33 / 34
Progress:  34 / 34
Interpolated:  34 TEC points in Series Runtime:  0:00:23.340901  s


In [4]:
alpha = 0.8
beta  = 0.7
# factor = alpha*beta
factor = 1

In [5]:
tec_CS2 = factor*tec_GPS
f = 13.575e9
tecu = 1e16
delta_iono_our = (40.3/f**2)* tec_CS2 * tecu
rads_error = sla_corrected-sla_uncorrected
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(rads_error, delta_iono_our)
mse

0.00014812495639694972

In [6]:
from sklearn.metrics import mean_squared_error
import scipy as sp

def fun(factor):
    # constants 
    tec_SAT = factor*tec_GPS
    tecu = 1e16
    delta_iono_our = (40.3/f1**2)* tec_SAT * tecu
    rads_error = sla_corrected-sla_uncorrected
    mse = mean_squared_error(rads_error, delta_iono_our)
    return mse

bnds = ((0.0, 1.0),)

sp.optimize.minimize(fun, (0.8), method='COBYLA', bounds=bnds)

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 1.2486926270626888e-05
       x: [ 9.339e-01]
    nfev: 21
   maxcv: 0.0

In [64]:
bnds = ((0.0, 1.0), )
lb, up = zip(*bnds)